# **Amazon Sentiment Analysis**

## Install Required Library for Data

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.9/73.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.3/235.3 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.5/213.5 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.0/349.0 kB 28.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.7.0
    Uninstalling fsspec-2025.7.0:
      Successfully uninstalled fsspec-2025.7.0


## **Importing Reruired Library**

In [3]:
from datasets import load_dataset
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score

warnings.filterwarnings('ignore')

## Loading Amazon Polarity DataSet

In [ ]:
dataset = load_dataset("amazon_polarity")
dataset

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00004.parquet:   0%|          | 0.00/260M [00:00<?, ?B/s]

train-00001-of-00004.parquet:   0%|          | 0.00/258M [00:00<?, ?B/s]

train-00002-of-00004.parquet:   0%|          | 0.00/255M [00:00<?, ?B/s]

train-00003-of-00004.parquet:   0%|          | 0.00/254M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3600000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/400000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'title', 'content'],
        num_rows: 3600000
    })
    test: Dataset({
        features: ['label', 'title', 'content'],
        num_rows: 400000
    })
})

### Converting DataSet Into DataFrame

In [ ]:
df_train = pd.DataFrame(dataset['train'])
df_test = pd.DataFrame(dataset['test'])

In [ ]:
df = pd.concat([df_train,df_test],ignore_index=True)

In [ ]:
df.head()

,label,title,content
0,1,Stuning even for the non-gamer,This sound track was beautiful! It paints the ...
1,1,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...
2,1,Amazing!,This soundtrack is my favorite music of all ti...
3,1,Excellent Soundtrack,I truly like this soundtrack and I enjoy video...
4,1,"Remember, Pull Your Jaw Off The Floor After He...","If you've played the game, you know how divine..."


## Data Preprocessing

In [ ]:
df.shape

(4000000, 3)

In [ ]:
df = df.sample(3000000, random_state=42)

In [ ]:
df.shape

(3000000, 3)

In [ ]:
df.isnull().sum()

,0
label,0
title,0
content,0


In [ ]:
df['text'] = df['title'] + " " + df['content']

In [ ]:
df = df[['label','text']]

In [ ]:
df.head()

,label,text
1049554,0,"Deeply disappointing, faulty morality & social..."
214510,1,insight into the philosophy of libertarian soc...
2145764,1,"a great book ""In vain did the Bedouins strive ..."
2198867,0,"toys for great sex wow, that was bad, I threw ..."
1184366,1,i love this movie!!!! i just finished reading ...


In [ ]:
df['label'].unique()

array([0, 1])

## **God Code For NLP Text Preprocessing**

In [ ]:
import re
import string
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def text_preprocess(text):
    # Remove emojis
    text = text.encode('ascii', 'ignore').decode('ascii')

    # Lowercase
    text = text.lower()

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])

    return text
df['text'] = df['text'].apply(text_preprocess)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## **Model Building**

### Spliting Data Into X and y Variables

In [ ]:
X = df['text']
y = df['label']

### Spliting Data Into Train Set And Test Set

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
bow_vectorizer = CountVectorizer()

In [ ]:
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)

### **Naive Byes**

In [ ]:
model_NB = MultinomialNB()

In [ ]:
model_NB.fit(X_train_bow,y_train)

MultinomialNB()

In [ ]:
pred_np = model_NB.predict(X_test_bow)

In [ ]:
print('accuracy = ',accuracy_score(y_test,pred_np))

accuracy =  0.851355


In [ ]:
vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1, 2))
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

### **Support Vector Machine Model (SVM)**

In [ ]:
svm = LinearSVC()
svm.fit(X_train_vec, y_train)

LinearSVC()

In [ ]:
y_pred = svm.predict(X_test_vec)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.9062266666666666
              precision    recall  f1-score   support

           0       0.91      0.90      0.91    299992
           1       0.90      0.91      0.91    300008

    accuracy                           0.91    600000
   macro avg       0.91      0.91      0.91    600000
weighted avg       0.91      0.91      0.91    600000



## **Converting Model Into Pickel File**


---


*SVM -> amazon_review_model.pkl*

---

*vectorizer -> vectorizer.pkl*


---


In [ ]:
import joblib
joblib.dump(svm, 'amazon_review_model.pkl')

['amazon_review_model.pkl']

In [ ]:
joblib.dump(vectorizer, 'vectorizer.pkl')

['vectorizer.pkl']